In [1]:
import os
import sys
import time
import math
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import carla

In [2]:
#Configs
map_idx = 1
MAP = ['circut_spa', 'RaceTrack', 'RaceTrack2']

# Plan na notebook
* Określamy kierunki pomiędzy nimi (oryginalnymi punktami)
* Wybieramy punkty które mają być wyżej/niżej
* Wypłaszczamy je, lub wyrównujemy -> interpolacja liniowa między skrajnie wysokim i niskim
* Testujemy spawnując auto po drodze
* zapisujemy jako csv

# W programie
* Obrobione pliki program będzie zaczytywał licząc kierunki ad hoc. Wyppłaszczanie -> stopnie zamienić na radiany czy coś i wtedy wypłaszczać.


 

In [3]:
#Connecting to client
client = carla.Client('localhost', 2000)
client.set_timeout(10.0) # seconds

In [150]:
# Formuła ze złota
def direction_lookup(pointA, pointB):
    sin_alpha = pointB[1] - pointA[1]
    cos_alpha = pointB[0] - pointA[0]
    
    alpha = np.degrees(np.arctan2(sin_alpha, cos_alpha)) 
    
    return alpha

In [154]:
world = client.load_world(MAP[map_idx])
blueprint_library = world.get_blueprint_library()
vehicle = blueprint_library.filter('*aud*')[0]
spawn_points = world.get_map().get_spawn_points()
spectator = world.get_spectator()

if MAP[map_idx] is 'circut_spa':
    spawn_points.pop(82)
elif MAP[map_idx] is 'RaceTrack2':
    spawn_points.pop(153)
elif MAP[map_idx] is 'RaceTrack':
    spawn_points.pop(183)

In [155]:
yaws = []
for idx, sp in enumerate(spawn_points[1:]):
    yaw = direction_lookup((spawn_points[idx].location.x, spawn_points[idx].location.y),
                           (sp.location.x, sp.location.y))
    yaws.append(yaw)
    
yaw = direction_lookup((spawn_points[-1].location.x, spawn_points[-1].location.y),
                       (spawn_points[0].location.x, spawn_points[0].location.y))
yaws.append(yaw)
    
len(yaws)

183

# Vehicle check

In [156]:
z = []

# i = np.random.randint(0, high=len(spawn_points))
# i = 150

# for idx, (spawn, yaw) in enumerate(zip(spawn_points[i:i+6], yaws[i:i+6])):
for idx, (spawn, yaw) in enumerate(zip(spawn_points, yaws)):
    location = spawn.location
    rotation = carla.Rotation(spawn.rotation.pitch, float(yaw), spawn.rotation.roll)
    spawn = carla.Transform(location, rotation)
    actor = world.spawn_actor(vehicle, spawn)
    
    if idx % 3 == 0:
        spec_loc = carla.Location(spawn.location.x, spawn.location.y, spawn.location.z+60)
        spec_rot = carla.Rotation(spawn.rotation.pitch-70., float(yaw), 0.)
        spectator.set_transform(carla.Transform(spec_loc, spec_rot))
    
    time.sleep(1.5)
    actor_z = actor.get_location().z
    z.append(actor_z+1.5)
    
    print(f'Vehicle {idx}, {yaw}, {actor_z+1.5}')
    actor.destroy()

Vehicle 0, 143.29125679163002, 2.2244279384613037
Vehicle 1, 132.65312658711284, 2.1516271829605103
Vehicle 2, 110.17065421392803, 2.0854029059410095
Vehicle 3, 90.0, 2.05164235830307
Vehicle 4, 73.05467418122463, 2.0458688139915466
Vehicle 5, 62.916416917854384, 2.02740740776062
Vehicle 6, 48.052921648207366, 2.0283931493759155
Vehicle 7, 43.555896680431815, 1.9736677408218384
Vehicle 8, 38.86774267341589, 1.9453035295009613
Vehicle 9, 34.18061642622044, 1.9196895360946655
Vehicle 10, 30.311233275071906, 1.9690799117088318
Vehicle 11, 13.695855524852535, 2.0743234753608704
Vehicle 12, 1.507449876063415, 2.0579065084457397
Vehicle 13, -11.821520049702086, 2.0487489700317383
Vehicle 14, -23.025454983035022, 2.035451889038086
Vehicle 15, -33.310665715364955, 2.008920192718506
Vehicle 16, -40.65677188715799, 1.9781647324562073
Vehicle 17, -52.5494155324237, 1.9279887676239014
Vehicle 18, -59.56026733118755, 1.8352992832660675
Vehicle 19, -64.73271237897585, 1.6992302685976028
Vehicle 20, 

In [157]:
spawn_points_df = pd.DataFrame().from_dict({idx: (sp.location.x, sp.location.y, sp.location.z)
                                            for idx, sp in enumerate(spawn_points)}).T

spawn_points_df.rename(columns={0:'x', 1:'y', 2:'z'}, inplace=True)
spawn_points_df['yaw'] = yaws
spawn_points_df['z'] = z
spawn_points_df.shape

spawn_points_df.to_csv(f'../data/spawn_points/{MAP[map_idx]}.csv', index=False)
len(spawn_points)

183